In [35]:
%load_ext autoreload
%autoreload 2
from proj1_helpers import *
from implementations import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
data_path = "data/train.csv"

In [28]:
y, x, ids = load_csv_data(data_path, sub_sample=False)

In [29]:
x_tr, x_te, y_tr, y_te = split_data(x, y, ratio=0.8, seed=1)

In [30]:
x_tr, mean_x_tr, std_x_tr = standardize(x_tr)
x_te, mean_x_te, std_x_te = standardize(x_te)

In [31]:
#SGD parameters
initial_w = np.zeros((x.shape[1]))
max_iters = 400
gamma = 0.05
lambda_ = 0.7

In [32]:
%%time
w_cv, loss_cv = perform_cross_validation(y_tr, x_tr,
                                         compute_reg_logistic_regression_loss,
                                         compute_reg_logistic_regression_gradient,
                                         max_iters=100)

CPU times: user 16min 47s, sys: 57.6 s, total: 17min 45s
Wall time: 4min 26s


In [37]:
y_pred = predict_labels_logistic_regression(w_cv, x_te)

In [38]:
accuracy = np.mean(np.equal(y_pred, y_te))

print(f'accuracy on test set : {accuracy}')

accuracy on test set : 0.6554


In [40]:
test_path = 'data/test.csv'

y_test, x_test, ids_test = load_csv_data(test_path)
x_test, mean_x_test, std_x_test = standardize(x_test)
y_test_pred = predict_labels_logistic_regression(w_cv, x_test)
create_csv_submission(ids_test, y_test_pred, 'reg_logistic.csv')